In [5]:
import pandas as pd
import numpy as np

#PnL_Spot_T----------------------------------------------
df = pd.read_excel(r"C:\RWE event\spot_prices_simulation_example.xlsx")
spot_values = df.iloc[:, 1].values[:]  # Assuming the second column contains spot values
spot_val = df.iloc[:, 1].values[1:]
PnL_Spot_T = sum((3.4 / 1000) * (-9 + spot) for spot in spot_val)
#The spot price model follows the one factor model of Schwartz (1997)
#PnL_OTC-------------------------------------------------

def calculate_forward_prices(S, T_values, kappa, alpha, sigma):
    forward_prices = {}
    
    for T in T_values:
        term1 = np.exp(-kappa * T) * np.log(S)
        term2 = (1 - np.exp(-kappa * T)) * alpha
        term3 = (sigma**2 * (1 - np.exp(-2 * kappa * T))) / (4 * kappa)
        
        forward_price = np.exp(term1 + term2 + term3)
        forward_prices[T] = forward_price
    
    return forward_prices
# Extract forward prices
S = 10  # Spot price used for forward price calculation
T_values = list(range(1, 37))  # Time to maturity values from 1 to 36
kappa = 0.15  # Parameter
alpha = 2.019899167  # Parameter
sigma = 0.2945  # Volatility parameter
yannual = 0.05  # Annual yield
T = 36  # Time to maturity

forward_prices_OTC = calculate_forward_prices(S, T_values, kappa, alpha, sigma)
# Calculate PnL^Spot_T using forward_prices
PnL_exchange_T = sum((3.4 / 1000) * (-9 + forward_prices_OTC[T]) for T in T_values)
PnL_OTC_T = PnL_exchange_T  #for no counter party default case

#cashbalance_T------------------------------------------------------------
forward_prices = {}
for i in range(len(spot_values)):
    forward_prices[i] = calculate_forward_prices(spot_values[i], T_values, kappa, alpha, sigma)

def calculate_cash_balance(yannual, forward_prices, T, spot_values):
    cashbalance = 0
    
    for t in range(1, T+1):
        inner_sum = 0
        I = 36 - t + 1
        
        for i in range(1, I+1):
            xa = i - 1
        
            
            if xa == 0:
                forward_diff = spot_values[t] - forward_prices[t-1][i]
            else:
                forward_diff = forward_prices[t][xa] - forward_prices[t-1][i]
                
            inner_sum += (np.exp(yannual/12) - 1) * max(forward_diff, 0)
        
        cashbalance += inner_sum
    
    cashbalance *= 3.4 / 1000
    return cashbalance

# Calculate cash balance
cashbalance_T = calculate_cash_balance(yannual, forward_prices, T, spot_values)

#PnL_total------------------------------------

#randomize hedging strategy----
import random

def generate_random_values():
    while True:
        x = random.uniform(0, 1)
        y = random.uniform(0, 1 - x)
        z = 1 - x - y
        
        if 0 <= x <= 1 and 0 <= y <= 1 and 0 <= z <= 1:
            break
    
    return x, y, z



weight_spot, weight_exchange, weight_OTC = generate_random_values()


# Calculate PnL_total using the generated weights
PnL_total = (weight_spot * PnL_Spot_T) + (weight_exchange * (PnL_exchange_T - cashbalance_T)) + (weight_OTC * PnL_OTC_T)


# Print the calculated PnL_total and weight values
print(f"PnL_total: {PnL_total}")
print(f"weight_spot: {weight_spot:.2f}")
print(f"weight_OTC: {weight_OTC:.2f}")
print(f"weight_exchange: {weight_exchange:.2f}")
print(f"PnL_Spot_T: {PnL_Spot_T}")
print(f"PnL_exchange_T: {PnL_exchange_T}")
print(f"cashbalance_T: {cashbalance_T}")
print(f"PnL_OTC_T: {PnL_OTC_T}")


PnL_total: 0.3042791536239401
weight_spot: 0.50
weight_OTC: 0.39
weight_exchange: 0.11
PnL_Spot_T: 0.6012504323912
PnL_exchange_T: 0.005425992844485863
cashbalance_T: 0.005875559259124063
PnL_OTC_T: 0.005425992844485863
